In [4]:
%%configure -f
{
"conf":{
    "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    },
    "jars":[
        "s3://mxtdw-audit-gamma/jars/iceberg-spark3-runtime.jar"
    ]
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1718585345363_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1718585345363_0006,pyspark,idle,Link,Link,None,✔


In [5]:
from pyspark.sql import SparkSession
catalog_name = "glue_catalog"
bucket_name = "mxtdw-audit-gamma"
bucket_prefix = "demo_iceberg/"
database_name = "playground"
warehouse_path = f"s3://{bucket_name}/{bucket_prefix}"
spark = SparkSession.builder \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", f"{warehouse_path}") \
    .config(f"spark.sql.catalog.{catalog_name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Creating the iceberg table 

In [7]:
%%sql 
create table glue_catalog.playground.demo(
    name string,
    phone string,
    email string,
    postalZip long,
    number int,
    country string,
    ingestion_date date
)
using iceberg 
PARTITIONED BY (country,year(ingestion_date))
Location 's3://mxtdw-audit-gamma/demo_iceberg/'
TBLPROPERTIES ( 
    'write.metadata.metrics.column.postalZip'='full',
    'write.metadata.metrics.column.country'='full',
    'write.metadata.metrics.column.ingestion_date'='full'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

Iceberg Metadata Queries 
Extended list can be found here - https://iceberg.apache.org/docs/nightly/spark-queries/

Getting the s3 files 

In [10]:
%%sql 
select * from glue_catalog.playground.demo.files;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

Getting the Snapshots

In [11]:
%%sql 
select * from glue_catalog.playground.demo.snapshots;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

Inserting the data 

In [14]:
import pyspark.sql.functions as f
df = spark.read.csv("s3://mxtdw-audit-gamma/demo_data/data.csv",header=True,inferSchema=True)
df = df.withColumn("ingestion_date",f.current_date())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+--------------------+---------+------+------------------+--------------+
|           name|         phone|               email|postalZip|number|           country|ingestion_date|
+---------------+--------------+--------------------+---------+------+------------------+--------------+
|   Price Savage|(424) 562-4115|varius.ultrices@g...|   586986|     7|             India|    2024-06-22|
|   Odette Logan|1-562-786-9778|ultricies.ligula@...|     1061|     8|Russian Federation|    2024-06-22|
|Katell Buchanan|(935) 445-9637|magna@protonmail.net|    55526|     9|           Nigeria|    2024-06-22|
|    Leigh Young|1-808-725-8983|ultrices.posuere....|     4906|     7|            Poland|    2024-06-22|
|  Bianca Sexton|1-383-310-3697|risus.at@outlook.net|   115771|     0|           Vietnam|    2024-06-22|
+---------------+--------------+--------------------+---------+------+------------------+--------------+

In [16]:
df.writeTo("glue_catalog.playground.demo").append()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Querying the Iceberg Table 

In [17]:
%%sql 
select * from glue_catalog.playground.demo;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [25]:
spark.sql("select * from glue_catalog.playground.demo.metadata_log_entries").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-----------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|timestamp              |file                                                                                                 |latest_snapshot_id |latest_schema_id|latest_sequence_number|
+-----------------------+-----------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|2024-06-22 20:14:45.207|s3://mxtdw-audit-gamma/demo_iceberg/metadata/00000-54f3d2dc-8b86-4b24-9ecd-741034ae757b.metadata.json|null               |null            |null                  |
|2024-06-22 20:38:45.767|s3://mxtdw-audit-gamma/demo_iceberg/metadata/00001-90aee3a9-521d-4f19-a1df-6d1eddb9aa64.metadata.json|3713858578440833675|0               |1                     |
+-----------------------+-----------------------------------

In [23]:
spark.sql("select * from glue_catalog.playground.demo.files").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+------------------------+------------+------------------+---------------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+----------------+------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
spark.sql("select * from glue_catalog.playground.demo.snapshots").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------------------+---------+---------+-----------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id|operation|manifest_list                                                                                                    |summary                                                                                                                                                                                                                                                                                                      |
+-----------------------+-----------

In [28]:
spark.read.format("avro").load("s3://mxtdw-audit-gamma/demo_iceberg/metadata/snap-3713858578440833675-1-99bb0c2c-45c5-47a8-bafa-ba824594195f.avro").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+----------------------+-------------------------+------------------------+----------------+-------------------+------------------+------------------------------------------------------------+
|manifest_path                                                                            |manifest_length|partition_spec_id|content|sequence_number|min_sequence_number|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_rows_count|existing_rows_count|deleted_rows_count|partitions                                                  |
+-----------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+----------------------+-------------------------+

In [30]:
spark.read.format("avro").load("s3://mxtdw-audit-gamma/demo_iceberg/metadata/99bb0c2c-45c5-47a8-bafa-ba824594195f-m0.avro").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------------+---------------+--------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|status|snapshot_id        |sequence_number|file_sequence_number|data_file                                                                                                                                                                                                      

Running Update command

In [31]:
spark.sql("""
    update glue_catalog.playground.demo set postalZip = 75234 
    where number =7;
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [32]:
spark.sql("select * from glue_catalog.playground.demo").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+----------------------------------+---------+------+------------------+--------------+
|name           |phone         |email                             |postalZip|number|country           |ingestion_date|
+---------------+--------------+----------------------------------+---------+------+------------------+--------------+
|Price Savage   |(424) 562-4115|varius.ultrices@google.net        |75234    |7     |India             |2024-06-22    |
|Leigh Young    |1-808-725-8983|ultrices.posuere.cubilia@yahoo.com|75234    |7     |Poland            |2024-06-22    |
|Katell Buchanan|(935) 445-9637|magna@protonmail.net              |55526    |9     |Nigeria           |2024-06-22    |
|Odette Logan   |1-562-786-9778|ultricies.ligula@icloud.net       |1061     |8     |Russian Federation|2024-06-22    |
|Bianca Sexton  |1-383-310-3697|risus.at@outlook.net              |115771   |0     |Vietnam           |2024-06-22    |
+---------------+--------------+----------------

In [36]:
spark.sql("select * from glue_catalog.playground.demo.snapshots order by committed_at desc").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------------------+-------------------+---------+-----------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                    |summary                                                                                                                                                                                                                                              

In [33]:
spark.sql("select * from glue_catalog.playground.demo.metadata_log_entries").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-----------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|timestamp              |file                                                                                                 |latest_snapshot_id |latest_schema_id|latest_sequence_number|
+-----------------------+-----------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|2024-06-22 20:14:45.207|s3://mxtdw-audit-gamma/demo_iceberg/metadata/00000-54f3d2dc-8b86-4b24-9ecd-741034ae757b.metadata.json|null               |null            |null                  |
|2024-06-22 20:38:45.767|s3://mxtdw-audit-gamma/demo_iceberg/metadata/00001-90aee3a9-521d-4f19-a1df-6d1eddb9aa64.metadata.json|3713858578440833675|0               |1                     |
|2024-06-22 21:21:19.331|s3://mxtdw-audit-gamma/demo_iceberg

In [51]:
spark.sql("select * from glue_catalog.playground.demo.files").show(truncate=False)
#if you take any files from this list , it will only give the updated files which contains the updated data 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+------------------------+------------+------------------+---------------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+----------------+------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Reading Snapshots

In [57]:
spark.sql("select * from glue_catalog.playground.demo version as of 3713858578440833675").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+--------------------+---------+------+------------------+--------------+
|           name|         phone|               email|postalZip|number|           country|ingestion_date|
+---------------+--------------+--------------------+---------+------+------------------+--------------+
|   Price Savage|(424) 562-4115|varius.ultrices@g...|   586986|     7|             India|    2024-06-22|
|Katell Buchanan|(935) 445-9637|magna@protonmail.net|    55526|     9|           Nigeria|    2024-06-22|
|    Leigh Young|1-808-725-8983|ultrices.posuere....|     4906|     7|            Poland|    2024-06-22|
|   Odette Logan|1-562-786-9778|ultricies.ligula@...|     1061|     8|Russian Federation|    2024-06-22|
|  Bianca Sexton|1-383-310-3697|risus.at@outlook.net|   115771|     0|           Vietnam|    2024-06-22|
+---------------+--------------+--------------------+---------+------+------------------+--------------+

In [58]:
spark.sql("select * from glue_catalog.playground.demo version as of 4680088034666527875").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+--------------------+---------+------+------------------+--------------+
|           name|         phone|               email|postalZip|number|           country|ingestion_date|
+---------------+--------------+--------------------+---------+------+------------------+--------------+
|Katell Buchanan|(935) 445-9637|magna@protonmail.net|    55526|     9|           Nigeria|    2024-06-22|
|   Odette Logan|1-562-786-9778|ultricies.ligula@...|     1061|     8|Russian Federation|    2024-06-22|
|  Bianca Sexton|1-383-310-3697|risus.at@outlook.net|   115771|     0|           Vietnam|    2024-06-22|
|   Price Savage|(424) 562-4115|varius.ultrices@g...|    75234|     7|             India|    2024-06-22|
|    Leigh Young|1-808-725-8983|ultrices.posuere....|    75234|     7|            Poland|    2024-06-22|
+---------------+--------------+--------------------+---------+------+------------------+--------------+

## CDC of tables , which can be used to stream the events 

#### Based on call columns 

In [69]:
%%sql
CALL glue_catalog.system.create_changelog_view(
  table => 'playground.demo',
  options => map('start-snapshot-id','3713858578440833675','end-snapshot-id', '4680088034666527875'),
  changelog_view => 'change_load'
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [77]:
spark.sql("SELECT * FROM change_load order by number desc , _change_type desc , country desc").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+--------------------+---------+------+-------+--------------+------------+---------------+-------------------+
|        name|         phone|               email|postalZip|number|country|ingestion_date|_change_type|_change_ordinal|_commit_snapshot_id|
+------------+--------------+--------------------+---------+------+-------+--------------+------------+---------------+-------------------+
| Leigh Young|1-808-725-8983|ultrices.posuere....|    75234|     7| Poland|    2024-06-22|      INSERT|              0|4680088034666527875|
|Price Savage|(424) 562-4115|varius.ultrices@g...|    75234|     7|  India|    2024-06-22|      INSERT|              0|4680088034666527875|
| Leigh Young|1-808-725-8983|ultrices.posuere....|     4906|     7| Poland|    2024-06-22|      DELETE|              0|4680088034666527875|
|Price Savage|(424) 562-4115|varius.ultrices@g...|   586986|     7|  India|    2024-06-22|      DELETE|              0|4680088034666527875|
+------------+------